In [2]:
import numpy as np
import h5py

import e2c as e2c_util

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
def reconstruction_loss(x, t_decoded):
    '''Reconstruction loss for the plain VAE'''
    v = 0.1
    # return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v) + 0.5*K.log(2*np.pi*v), axis=-1))
    return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v), axis=-1))
    # return K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2, axis=-1)


def kl_normal_loss(qm, q_logv, pm, p_logv):
    # 0.5 * (torch.log(pv) - torch.log(qv) + qv / pv + (qm - pm).pow(2) / pv - 1)
    # -0.5 * K.sum(1 + t_log_var - K.square(t_mean) - K.exp(t_log_var), axis=-1)
    kl = -0.5 * (1 - p_logv + q_logv - K.exp(q_logv) / K.exp(p_logv) - K.square(qm - pm) / K.exp(p_logv))
    return K.mean(K.sum(kl, axis=-1))


def get_flux_loss(m, state, state_pred):
    # state, state_pred shape (batch_size, 60, 60, 2)
    # p, p_pred shape (batch_size, 60, 60, 1)
    # k shape (batch_size, 60, 60, 1)
    
    # Only consider discrepancies in total flux, not in phases (saturation not used) 
    
    perm = K.exp(m)
#     print(K.int_shape(perm))
#     print(K.int_shape(state))
#     print(K.int_shape(state_pred))
    p = K.expand_dims(state[:, :, :, 1], -1)
    p_pred = K.expand_dims(state_pred[:, :, :, 1], -1)
#     print(K.int_shape(p))
#     print(K.int_shape(p_pred))
          
    
    tran_x = 1./perm[:, 1:, ...] + 1./perm[:, :-1, ...]
    tran_y = 1./perm[:, :, 1:, ...] + 1./perm[:, :, :-1, ...]
#     print(K.int_shape(tran_x))
#     print(K.int_shape(tran_y))
    flux_x = (p[:, 1:, ...] - p[:, :-1, ...]) / tran_x
    flux_y = (p[:, :, 1:, :] - p[:, :, :-1, :]) / tran_y
    flux_x_pred = (p_pred[:, 1:, ...] - p_pred[:, :-1, ...]) / tran_x
    flux_y_pred = (p_pred[:, :, 1:, :] - p_pred[:, :, :-1, :]) / tran_y

    loss_x = K.sum(K.abs(K.batch_flatten(flux_x) - K.batch_flatten(flux_x_pred)), axis=-1)
    loss_y = K.sum(K.abs(K.batch_flatten(flux_y) - K.batch_flatten(flux_y_pred)), axis=-1)

    loss_flux = K.mean(loss_x + loss_y)
    return loss_flux

In [4]:
def create_e2c(latent_dim, u_dim, input_shape):
    '''
    Creates a E2C.

    Args:
        latent_dim: dimensionality of latent space
        return_kl_loss_op: whether to return the operation for
                           computing the KL divergence loss.

    Returns:
        The VAE model. If return_kl_loss_op is True, then the
        operation for computing the KL divergence loss is
        additionally returned.
    '''

    encoder_ = e2c_util.create_encoder(latent_dim, input_shape)
    decoder_ = e2c_util.create_decoder(latent_dim, input_shape)
    transition_ = e2c_util.create_trans(latent_dim, u_dim)
    sampler_ = e2c_util.create_sampler()

    return encoder_, decoder_, transition_, sampler_

In [5]:
# Create plain E2C model and associated loss operations

################### case specification ######################

#     data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP/'
data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP_RATE/'
output_dir = '/data3/Astro/lstm_rom/e2c_larry/saved_models/'

#     case_name = '9w_bhp'
case_name = '9w_bhp_rate'

#     case_suffix = '_single_out_rel_2'
case_suffix = '_fix_wl_rel_1'
#     case_suffix = '_single_out_rel_3'
train_suffix = '_with_p'
model_suffix = '_flux_loss'


train_file = case_name + '_e2c_train' + case_suffix + train_suffix + '_n6600_dt20day_nt22_nrun300.mat'
eval_file = case_name + '_e2c_eval' + case_suffix + train_suffix +'_n2200_dt20day_nt22_nrun100.mat'

#################### model specification ##################
epoch = 10
batch_size = 4
learning_rate = 1e-4
latent_dim = 50
u_dim = 9 # control dimension

# load data
hf_r = h5py.File(data_dir + train_file, 'r')
state_t_train = np.array(hf_r.get('state_t'))
state_t1_train = np.array(hf_r.get('state_t1'))
bhp_train = np.array(hf_r.get('bhp'))
hf_r.close()

num_train = state_t_train.shape[0]

hf_r = h5py.File(data_dir + eval_file, 'r')
state_t_eval = np.array(hf_r.get('state_t'))
state_t1_eval = np.array(hf_r.get('state_t1'))
bhp_eval = np.array(hf_r.get('bhp'))
hf_r.close()

m = np.loadtxt("/data/cees/zjin/lstm_rom/sim_runs/case4_9w_bhp_rate/template/logk1.dat")
m = m.reshape(60,60,1)
print('m shape is ', m.shape)
#     m_tf = K.placeholder((batch_size, 60, 60 ,1))
m_tf = Input(shape=(60, 60, 1))



m_eval = np.repeat(np.expand_dims(m, axis = 0), state_t_eval.shape[0], axis = 0)
print("m_eval shape is ", m_eval.shape)
m = np.repeat(np.expand_dims(m,axis = 0), state_t_train.shape[0], axis = 0)
print("m shape is ", m.shape)


m shape is  (60, 60, 1)
m_eval shape is  (2200, 60, 60, 1)
m shape is  (6600, 60, 60, 1)


In [6]:

# Construct E2C
input_shape = (60, 60, 2)
encoder, decoder, transition, sampler = create_e2c(latent_dim, u_dim, input_shape)

xt = Input(shape=input_shape)
xt1 = Input(shape=input_shape)
ut = Input(shape=(u_dim, ))

zt_mean, zt_logvar = encoder(xt)
zt = sampler([zt_mean, zt_logvar])
xt_rec = decoder(zt)

zt1_mean, zt1_logvar = encoder(xt1)

# zt1_pred, zt1_mean_pred, zt1_logvar_pred = transition([zt, ut])
zt1_pred, zt1_mean_pred = transition([zt, zt_mean, ut])
xt1_pred = decoder(zt1_pred)

# Compute loss
loss_rec_t = reconstruction_loss(xt, xt_rec)
loss_rec_t1 = reconstruction_loss(xt1, xt1_pred)

loss_flux_t = get_flux_loss(m_tf, xt, xt_rec) / 300.
loss_flux_t1 = get_flux_loss(m_tf, xt1, xt1_pred) / 300.

loss_kl = kl_normal_loss(zt_mean, zt_logvar, 0., 0.)  # log(1.) = 0.
loss_bound = loss_rec_t + loss_rec_t1 + loss_kl  + loss_flux_t + loss_flux_t1

# loss_trans = kl_normal_loss(zt1_mean_pred, zt1_logvar_pred, zt1_mean, zt1_logvar)

# Use zt_logvar to approximate zt1_logvar_pred
loss_trans = kl_normal_loss(zt1_mean_pred, zt_logvar, zt1_mean, zt1_logvar)


trans_loss_weight = 1.0 # lambda in E2C paper Eq. (11)
loss = loss_bound + trans_loss_weight * loss_trans



In [7]:
# Optimization
opt = Adam(lr=learning_rate)

trainable_weights = encoder.trainable_weights + decoder.trainable_weights + transition.trainable_weights

updates = opt.get_updates(loss, trainable_weights)

iterate = K.function([xt, ut, xt1, m_tf], [loss, loss_rec_t, loss_rec_t1, loss_kl, loss_trans, loss_flux_t, loss_flux_t1], updates=updates)

eval_loss = K.function([xt, ut, xt1, m_tf], [loss])

num_batch = int(num_train/batch_size)

for e in range(epoch):
    for ib in range(num_batch):
        ind0 = ib * batch_size
        state_t_batch = state_t_train[ind0:ind0+batch_size, ...]
        state_t1_batch = state_t1_train[ind0:ind0 + batch_size, ...]
        bhp_batch = bhp_train[ind0:ind0 + batch_size, ...]
        m_batch = m[ind0:ind0 + batch_size, ...]

        output = iterate([state_t_batch, bhp_batch, state_t1_batch, m_batch])

        # tf.session.run(feed_dict={xt: sat_t_batch, ut: bhp_batch, xt1: sat_t1_batch}, ...
        #                fetches= [loss, loss_rec_t, loss_rec_t1, loss_kl, loss_trans, updates])
        # But output tensor for the updates operation is not returned

        if ib % 10 == 0:
            print('Epoch %d/%d, Batch %d/%d, Loss %f, Loss rec %f, loss rec t1 %f, loss kl %f, loss_trans %f, loss flux %f, loss flux t1 %f'
                  % (e+1, epoch, ib+1, num_batch, output[0], output[1], output[2], output[3], output[4], output[5], output[6]))
    eval_loss_val = eval_loss([state_t_eval, bhp_eval, state_t1_eval, m_eval])

    print('Epoch %d/%d, Train loss %f, Eval loss %f' % (e + 1, epoch, output[0], eval_loss_val[0]))


encoder.save_weights(output_dir + 'e2c_encoder_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
decoder.save_weights(output_dir + 'e2c_decoder_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
transition.save_weights(output_dir + 'e2c_transition_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                        % (num_train, latent_dim, learning_rate, epoch))

Epoch 1/10, Batch 1/1650, Loss 4038.392822, Loss rec 1701.444824, loss rec t1 2089.495361, loss kl 0.031140, loss_trans 0.026130, loss flux 119.678650, loss flux t1 127.716484
Epoch 1/10, Batch 11/1650, Loss 2554.875977, Loss rec 741.356750, loss rec t1 853.260620, loss kl 15.125269, loss_trans 12.703239, loss flux 509.874451, loss flux t1 422.555786
Epoch 1/10, Batch 21/1650, Loss 1557.361450, Loss rec 484.480804, loss rec t1 536.387817, loss kl 9.275908, loss_trans 6.374589, loss flux 241.584961, loss flux t1 279.257263
Epoch 1/10, Batch 31/1650, Loss 1112.858765, Loss rec 366.910950, loss rec t1 406.548187, loss kl 4.336070, loss_trans 3.413725, loss flux 160.765549, loss flux t1 170.884338
Epoch 1/10, Batch 41/1650, Loss 985.137329, Loss rec 346.660461, loss rec t1 354.429352, loss kl 3.759231, loss_trans 2.784351, loss flux 136.276382, loss flux t1 141.227524
Epoch 1/10, Batch 51/1650, Loss 762.994019, Loss rec 277.553040, loss rec t1 272.489685, loss kl 2.402936, loss_trans 1.947

Epoch 1/10, Batch 481/1650, Loss 232.191895, Loss rec 56.723797, loss rec t1 68.685555, loss kl 0.626647, loss_trans 0.249654, loss flux 54.859707, loss flux t1 51.046539
Epoch 1/10, Batch 491/1650, Loss 238.631195, Loss rec 62.441269, loss rec t1 74.228638, loss kl 0.437624, loss_trans 0.272431, loss flux 53.431232, loss flux t1 47.820007
Epoch 1/10, Batch 501/1650, Loss 269.780975, Loss rec 77.777695, loss rec t1 90.855621, loss kl 0.878233, loss_trans 0.538329, loss flux 52.102959, loss flux t1 47.628132
Epoch 1/10, Batch 511/1650, Loss 274.017029, Loss rec 78.837631, loss rec t1 91.634979, loss kl 0.671138, loss_trans 0.272636, loss flux 53.693245, loss flux t1 48.907394
Epoch 1/10, Batch 521/1650, Loss 227.295578, Loss rec 55.506752, loss rec t1 64.365829, loss kl 0.825911, loss_trans 0.386308, loss flux 53.951725, loss flux t1 52.259045
Epoch 1/10, Batch 531/1650, Loss 211.640305, Loss rec 53.571312, loss rec t1 62.653252, loss kl 0.455716, loss_trans 0.227345, loss flux 49.57048

Epoch 1/10, Batch 961/1650, Loss 146.064194, Loss rec 35.538204, loss rec t1 38.600197, loss kl 0.321387, loss_trans 0.089723, loss flux 37.333828, loss flux t1 34.180859
Epoch 1/10, Batch 971/1650, Loss 178.052399, Loss rec 47.816772, loss rec t1 53.721390, loss kl 0.388423, loss_trans 0.146861, loss flux 38.970829, loss flux t1 37.008121
Epoch 1/10, Batch 981/1650, Loss 174.705399, Loss rec 45.048820, loss rec t1 55.005657, loss kl 0.314856, loss_trans 0.088113, loss flux 39.183144, loss flux t1 35.064800
Epoch 1/10, Batch 991/1650, Loss 184.408203, Loss rec 42.857517, loss rec t1 53.632256, loss kl 0.271332, loss_trans 0.051972, loss flux 45.523632, loss flux t1 42.071480
Epoch 1/10, Batch 1001/1650, Loss 281.685455, Loss rec 70.934845, loss rec t1 89.173866, loss kl 0.645596, loss_trans 0.236516, loss flux 60.719219, loss flux t1 59.975430
Epoch 1/10, Batch 1011/1650, Loss 223.987015, Loss rec 62.893044, loss rec t1 71.636032, loss kl 0.249625, loss_trans 0.080540, loss flux 47.091

Epoch 1/10, Batch 1441/1650, Loss 156.319092, Loss rec 34.724789, loss rec t1 46.496399, loss kl 0.434294, loss_trans 0.127593, loss flux 36.658817, loss flux t1 37.877190
Epoch 1/10, Batch 1451/1650, Loss 129.998276, Loss rec 33.390976, loss rec t1 37.864182, loss kl 0.340138, loss_trans 0.084459, loss flux 29.798727, loss flux t1 28.519785
Epoch 1/10, Batch 1461/1650, Loss 123.025307, Loss rec 28.835087, loss rec t1 31.349075, loss kl 0.247718, loss_trans 0.034680, loss flux 33.105858, loss flux t1 29.452888
Epoch 1/10, Batch 1471/1650, Loss 141.009521, Loss rec 36.898380, loss rec t1 39.097115, loss kl 0.369869, loss_trans 0.085198, loss flux 32.308979, loss flux t1 32.249973
Epoch 1/10, Batch 1481/1650, Loss 146.943756, Loss rec 36.766670, loss rec t1 44.331429, loss kl 0.302128, loss_trans 0.075139, loss flux 33.076859, loss flux t1 32.391533
Epoch 1/10, Batch 1491/1650, Loss 137.574860, Loss rec 33.782700, loss rec t1 36.601387, loss kl 0.511071, loss_trans 0.136174, loss flux 34

Epoch 2/10, Batch 271/1650, Loss 116.362480, Loss rec 27.855495, loss rec t1 29.871208, loss kl 0.283315, loss_trans 0.056684, loss flux 29.786160, loss flux t1 28.509623
Epoch 2/10, Batch 281/1650, Loss 160.937546, Loss rec 48.069954, loss rec t1 54.782158, loss kl 0.235743, loss_trans 0.034059, loss flux 30.407257, loss flux t1 27.408367
Epoch 2/10, Batch 291/1650, Loss 117.535835, Loss rec 27.711464, loss rec t1 30.340364, loss kl 0.273659, loss_trans 0.045421, loss flux 31.168400, loss flux t1 27.996536
Epoch 2/10, Batch 301/1650, Loss 100.412651, Loss rec 25.282467, loss rec t1 26.629313, loss kl 0.177873, loss_trans 0.023204, loss flux 25.179243, loss flux t1 23.120548
Epoch 2/10, Batch 311/1650, Loss 91.624306, Loss rec 16.366505, loss rec t1 20.777315, loss kl 0.175471, loss_trans 0.007878, loss flux 28.194656, loss flux t1 26.102478
Epoch 2/10, Batch 321/1650, Loss 107.653397, Loss rec 23.887150, loss rec t1 25.759708, loss kl 0.266612, loss_trans 0.054308, loss flux 30.105822

...

Epoch 9/10, Batch 921/1650, Loss 30.087208, Loss rec 4.663772, loss rec t1 5.731049, loss kl 0.115541, loss_trans 0.004662, loss flux 9.948407, loss flux t1 9.623776
Epoch 9/10, Batch 931/1650, Loss 29.019234, Loss rec 4.949775, loss rec t1 5.535710, loss kl 0.117936, loss_trans 0.003516, loss flux 8.932229, loss flux t1 9.480068
Epoch 9/10, Batch 941/1650, Loss 31.030937, Loss rec 4.809843, loss rec t1 5.100045, loss kl 0.152912, loss_trans 0.004263, loss flux 10.928721, loss flux t1 10.035154
Epoch 9/10, Batch 951/1650, Loss 29.824123, Loss rec 4.637723, loss rec t1 5.115445, loss kl 0.099395, loss_trans 0.002629, loss flux 9.984298, loss flux t1 9.984634
Epoch 9/10, Batch 961/1650, Loss 33.268318, Loss rec 5.754157, loss rec t1 7.079749, loss kl 0.075690, loss_trans 0.002282, loss flux 10.741287, loss flux t1 9.615156
Epoch 9/10, Batch 971/1650, Loss 35.605042, Loss rec 7.014458, loss rec t1 7.743448, loss kl 0.082854, loss_trans 0.004598, loss flux 11.261799, loss flux t1 9.497887


Epoch 9/10, Batch 1411/1650, Loss 28.806301, Loss rec 5.263215, loss rec t1 5.732684, loss kl 0.121449, loss_trans 0.005364, loss flux 8.741448, loss flux t1 8.942142
Epoch 9/10, Batch 1421/1650, Loss 31.296179, Loss rec 5.350009, loss rec t1 6.232255, loss kl 0.109103, loss_trans 0.004612, loss flux 10.078309, loss flux t1 9.521891
Epoch 9/10, Batch 1431/1650, Loss 29.208961, Loss rec 4.906151, loss rec t1 5.215652, loss kl 0.116191, loss_trans 0.003998, loss flux 9.801683, loss flux t1 9.165287
Epoch 9/10, Batch 1441/1650, Loss 40.287483, Loss rec 6.701065, loss rec t1 7.000772, loss kl 0.166802, loss_trans 0.006416, loss flux 14.269510, loss flux t1 12.142919
Epoch 9/10, Batch 1451/1650, Loss 29.933016, Loss rec 5.717818, loss rec t1 6.484163, loss kl 0.116216, loss_trans 0.006690, loss flux 9.170836, loss flux t1 8.437292
Epoch 9/10, Batch 1461/1650, Loss 28.465673, Loss rec 4.724074, loss rec t1 5.376213, loss kl 0.100940, loss_trans 0.003478, loss flux 9.510464, loss flux t1 8.75

Epoch 10/10, Batch 251/1650, Loss 32.359665, Loss rec 6.578634, loss rec t1 6.585369, loss kl 0.127673, loss_trans 0.008139, loss flux 9.757561, loss flux t1 9.302290
Epoch 10/10, Batch 261/1650, Loss 28.979313, Loss rec 4.680577, loss rec t1 5.024070, loss kl 0.085749, loss_trans 0.002889, loss flux 9.889231, loss flux t1 9.296797
Epoch 10/10, Batch 271/1650, Loss 36.434017, Loss rec 5.859568, loss rec t1 6.748791, loss kl 0.132025, loss_trans 0.003574, loss flux 11.883625, loss flux t1 11.806434
Epoch 10/10, Batch 281/1650, Loss 31.007025, Loss rec 5.276945, loss rec t1 6.275037, loss kl 0.107053, loss_trans 0.004373, loss flux 10.092351, loss flux t1 9.251262
Epoch 10/10, Batch 291/1650, Loss 37.646790, Loss rec 6.350897, loss rec t1 7.678246, loss kl 0.136892, loss_trans 0.003467, loss flux 12.213326, loss flux t1 11.263963
Epoch 10/10, Batch 301/1650, Loss 30.194576, Loss rec 5.257105, loss rec t1 6.379216, loss kl 0.082223, loss_trans 0.002743, loss flux 9.468697, loss flux t1 9.

Epoch 10/10, Batch 741/1650, Loss 30.951706, Loss rec 5.076095, loss rec t1 5.731070, loss kl 0.109147, loss_trans 0.004960, loss flux 9.973817, loss flux t1 10.056617
Epoch 10/10, Batch 751/1650, Loss 30.734493, Loss rec 4.890276, loss rec t1 5.581216, loss kl 0.131386, loss_trans 0.004460, loss flux 10.322497, loss flux t1 9.804657
Epoch 10/10, Batch 761/1650, Loss 33.342224, Loss rec 4.797504, loss rec t1 5.779922, loss kl 0.161136, loss_trans 0.009652, loss flux 12.054098, loss flux t1 10.539913
Epoch 10/10, Batch 771/1650, Loss 28.516592, Loss rec 3.331267, loss rec t1 4.287283, loss kl 0.104300, loss_trans 0.003832, loss flux 11.022439, loss flux t1 9.767470
Epoch 10/10, Batch 781/1650, Loss 32.098164, Loss rec 5.441452, loss rec t1 6.611840, loss kl 0.093036, loss_trans 0.002770, loss flux 10.177156, loss flux t1 9.771911
Epoch 10/10, Batch 791/1650, Loss 34.228775, Loss rec 5.460433, loss rec t1 5.943467, loss kl 0.100200, loss_trans 0.005842, loss flux 11.484043, loss flux t1 

Epoch 10/10, Batch 1231/1650, Loss 28.187290, Loss rec 4.182763, loss rec t1 4.584816, loss kl 0.112985, loss_trans 0.003292, loss flux 9.572219, loss flux t1 9.731216
Epoch 10/10, Batch 1241/1650, Loss 31.351027, Loss rec 4.835472, loss rec t1 5.660447, loss kl 0.107257, loss_trans 0.003146, loss flux 10.390506, loss flux t1 10.354198
Epoch 10/10, Batch 1251/1650, Loss 30.980707, Loss rec 5.093291, loss rec t1 5.352796, loss kl 0.090541, loss_trans 0.003458, loss flux 10.514562, loss flux t1 9.926060
Epoch 10/10, Batch 1261/1650, Loss 33.682724, Loss rec 7.098945, loss rec t1 7.941061, loss kl 0.085998, loss_trans 0.004823, loss flux 9.478573, loss flux t1 9.073325
Epoch 10/10, Batch 1271/1650, Loss 33.293056, Loss rec 6.465392, loss rec t1 6.843697, loss kl 0.100345, loss_trans 0.006616, loss flux 10.289912, loss flux t1 9.587098
Epoch 10/10, Batch 1281/1650, Loss 26.815670, Loss rec 3.591741, loss rec t1 4.331963, loss kl 0.117606, loss_trans 0.003152, loss flux 9.672795, loss flux 